# Projeto de Previsão de Acidente Vascular Cerebral (AVC) com Python

Este notebook realiza a análise completa para a construção e avaliação de um modelo preditivo de AVC, seguindo a rubrica de Machine Learning para a área da saúde.

# Tarefa Um: Importação de dados e pré-processamento de dados

## Carregar dados e instalar pacotes

In [5]:
# Importação das bibliotecas essenciais para Ciência de Dados em Python

# Pandas: A biblioteca principal para carregar, manipular e limpar dados (nosso 'tidyverse' do Python)
import pandas as pd 
# NumPy: Usado para operações numéricas e arrays
import numpy as np 
# Matplotlib/Seaborn: Usados para visualização de dados (criação de gráficos). Seaborn é ótimo para estatística.
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn: É a biblioteca padrão para Machine Learning em Python
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE 
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Define o caminho do arquivo
file_path = '../data/healthcare-dataset-stroke-data.csv'

# Lê o arquivo CSV para um DataFrame do Pandas (df)
df = pd.read_csv(file_path)

# Mostra as primeiras 5 linhas para verificar se o carregamento funcionou
print("--- Primeiras 5 linhas do Dataset ---")
display(df.head())

--- Primeiras 5 linhas do Dataset ---


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


## Descreva e explore os dados

In [9]:
# Remove a coluna 'id' imediatamente se ela existir, embora tenhamos confirmado que ela não existe.
# O bloco 'try/except' garante que o código não quebre se a coluna não for encontrada (boa prática!)
try:
    df = df.drop('id', axis=1)
except KeyError:
    print("A coluna 'id' não foi encontrada, seguindo com o DataFrame original.")

# --- EXPLORAÇÃO INICIAL: Estrutura e Tipos de Dados ---
print("--- Estrutura e Tipos de Dados ---")
# .info() fornece um resumo: número de linhas, colunas e, o mais importante,
# o tipo de dado de cada coluna (float, int, object) e a contagem de valores não-nulos.
df.info()

# --- VERIFICAÇÃO DE VALORES AUSENTES (NA) ---
# O .isnull() verifica onde há NA (retorna True/False) e .sum() soma quantos há por coluna
print("\n--- Contagem de Valores Ausentes (NA) ---")
print(df.isnull().sum())

A coluna 'id' não foi encontrada, seguindo com o DataFrame original.
--- Estrutura e Tipos de Dados ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int64  
 3   heart_disease      5110 non-null   int64  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                4909 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 439.3+ KB

--- Contagem de Valores Ausentes (NA) ---
gender                 0
age                    0
hyperten

## Limpeza e Imputação de Dados

# Task Two: Build prediction models

## 4. Particionamento e Preparação dos Dados


## 5. Treinamento dos Modelos

# Task Three: Evaluate and select prediction models

# Task Four: Deploy the prediction model

# Task Five: Findings and Conclusions